In [1]:
import torch
import torch.nn as nn
import os
import torch.utils.data as data
from importlib import import_module
import matplotlib.pyplot as plt
import torch.optim as optim
import copy



vggish_input=import_module(".torchvggish.vggish_input","torchvggish-master_changed")
use_pretrained = True

#GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
#損失関数
#criterion = nn.CrossEntropyLoss()
#勾配計算手法?

#optimizer = optim.SGD(params_to_update, lr=0.01)

vggish/embedding:0
Using cuda device


In [2]:
#データセットの定義？
classes=["Speech","Music"]
dataset_dir="GetAudiosetSample/result"


def make_path_list():
    train_file_list=[]
    valid_file_list=[]
    for i in range(len(classes)):
        dir_name=os.path.join(dataset_dir,classes[i]).replace("\\","/")
        file_list=os.listdir(dir_name)
        
        #8割を学習用、残りを検証用にする
        num_data = len(file_list)
        num_split = int(num_data*0.8)
        
        train_file_list += [[os.path.join(dir_name, file).replace('\\', '/'), i] for file in file_list[:num_split] ]
        valid_file_list += [[os.path.join(dir_name, file).replace('\\', '/'), i] for file in file_list[num_split:]]
    return {"train":train_file_list,"valid":valid_file_list}
# 画像データへのファイルパスとラベルを格納したリストを取得する
path_dict = make_path_list()
## リストが変かもなのでみてみる


print('学習データファイル数 : ', len(path_dict["train"]))
##### 先頭3件だけ表示
print(path_dict["train"][:3])

print('検証データファイル数 : ', len(path_dict["valid"]))
##### 先頭3件だけ表示
print(path_dict["valid"][:3])

FileNotFoundError: [WinError 3] 指定されたパスが見つかりません。: 'GetAudiosetSample/result/Music'

In [ ]:

class MyDataset(data.Dataset):
    '''
    data_dictは[パス,番号]
    '''
    def __init__(self, path_dict,  phase='train'):
        #self.data_dict = data_dict
        self.data_dict = []
        for path, label in path_dict:
            for data in vggish_input.wavfile_to_examples(path):
                self.data_dict.append([data, label])
                
        
        self.phase = phase
        
    def __len__(self):
        return len(self.data_dict)
    
    def __getitem__(self,index):
        
        wav_data,label = self.data_dict[index]
        
        return wav_data, label

#DataSetを実際に作ってみる 

train_dataset = MyDataset(
    path_dict=path_dict["train"],
    phase="train"
)

valid_dataset = MyDataset(
    path_dict=path_dict["valid"],
    phase="valid"
)




In [ ]:
print("学習用データ数 : ", len(train_dataset))
print("検証用データ数 : ", len(valid_dataset))

In [ ]:
#dataloaderを用いてミニバッチを作成
def gen_dataloader_dict(batch_size=10):
    batch_size=10

    train_dataloader=data.DataLoader(
        train_dataset, batch_size = batch_size, shuffle=True
    )
    valid_dataloader=data.DataLoader(
        valid_dataset, batch_size = batch_size//2, shuffle=True
    )

    dataloader_dict={
        'train': train_dataloader, 
        'valid': valid_dataloader
    }
    return dataloader_dict


#print(dataloader_dict)

In [ ]:
class trainer:
    def __init__(self, _classfilter=nn.Sequential(
                nn.Linear(128,64),
                nn.Sigmoid(),
                nn.Linear(64,5),
                nn.Softmax(dim=1)
                ), 
                _criterion=nn.CrossEntropyLoss(),
                _optimizer = optim.SGD,#params_to_update, lr=0.01),
                _lr=0.01,
                 _weight_decay=0.0,
                _update_param_names=
                     ['classfilter.0.weight', 'classfilter.0.bias', 
                          'classfilter.2.weight','classfilter.2.bias'],
                _print=False
                ):
        '''
        _classfilter:クラスフィルタ層の中身をnn.Sequentialで宣言したものを入れる
            nn.Sequential(
                nn.Linear(128,64),
                nn.Sigmoid(),
                nn.Linear(64,5),
                nn.Softmax()
                )
        
        _num_epochs:学習時のエポック数
        
        _criterion:損失関数
        _optimizer:最適化手法
        _lr:学習の速度
        '''
        dbprint=(lambda x:print(x)) if _print else (lambda x:None)

        
        
        self.model = torch.hub.load('torchvggish-master_changed', 'vggish', source='local', preprocess=False,postprocess=False)
        self.model.classfilter=_classfilter

        
        self.model=self.model.to(device)
        self.model.eval()
        #学習させるパラメータを格納
        self.params_to_update=[]
        #対象以外は勾配計算をせず、変化しないようにもする
        for name,param in self.model.named_parameters():
            if name in _update_param_names:
                param.requires_grad = True
                self.params_to_update.append(param)
                #print("name : ",name)
            else:
                param.requires_grad = False
        
        self.lr=_lr
        self.weight_decay=_weight_decay
        self.criterion=_criterion
        self.optimizer=_optimizer(self.params_to_update, self.lr,self.weight_decay)
        dbprint(self.model)
                
    def train(self,dataloader_dict,_num_epochs=100,_print=False):
        dbprint=(lambda x:print(x)) if _print else (lambda x:None)
        #エポック数
        self.num_epochs=_num_epochs
        self.x=[]
        self.y=[]
        
        
        for epoch in range(self.num_epochs):
            dbprint('Epoch {}/{}'.format(epoch+1,self.num_epochs))

            for phase in ['train','valid']:
                if phase == 'train':
                    self.model.train()#学習モード
                else:
                    self.model.eval()#検証モード
                #epoch全体の損失の輪と正解数
                epoch_loss=0.0
                epoch_corrects=0
                count=0.0
                #dbprint(phase)
                for inputs, labels in dataloader_dict[phase]:
                    #入力の確認
                    #dbprint(len(inputs),len(labels))
                    #勾配計算を初期化する
                    self.optimizer.zero_grad()


                    with torch.set_grad_enabled(phase=='train'):
                        #labelsをcudaに
                        #dbprint(inputs.shape)
                        labels=labels.to(device)
                        inputs=inputs.to(device)
                        outputs=self.model(inputs)

                        #損失関数を計算
                        loss=self.criterion(outputs, labels)
                        #ラベルを予測
                        _,preds = torch.max(outputs,1)

                        #訓練時は逆伝搬の計算
                        if phase == "train":
                            #逆伝搬
                            loss.backward()

                            #パラメータ更新
                            self.optimizer.step()

                        #イテレーション結果の計算
                        #lossの合計を更新
                        #pytorchの使用上バッチ内の平均lossが計算されているのでデータ数をかけて合計にする
                        #損失和を「全データの損失/データ数」で求めるせいらしい?
                        #dbprint(len(inputs))
                        epoch_loss += loss.item() * inputs.size(0)

                        #正解数の合計を更新
                        epoch_corrects += torch.sum(preds == labels.data)

                #epochのlossと正解数の表示
                epoch_loss=epoch_loss/len(dataloader_dict[phase].dataset)
                epoch_acc=epoch_corrects.double()/len(dataloader_dict[phase].dataset)

                dbprint('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                self.x.append(epoch_acc)
                self.y.append(epoch)
    
    def graph(self,_title):
        plt.plot(self.y[::2],[i.tolist() for i in self.x][::2], label="train", color ="Green")
        plt.plot(self.y[::2],[i.tolist() for i in self.x][1::2], label="valid", color ="Blue")
        plt.legend(loc='upper left')
        plt.title(_title)
        plt.show()

In [ ]:

filters=[nn.Sequential(
        nn.Linear(128,64),
        nn.Sigmoid(),
#        nn.Linear(64,32),
#        nn.Sigmoid(),
        nn.Linear(64,5),
        nn.Softmax(dim=1)
    )
         #,
#    nn.Sequential(
#        nn.Linear(128,64),
#        nn.Sigmoid(),
#        nn.Linear(64,32),
#       nn.Sigmoid(),
#        nn.Linear(32,5),
#        nn.Softmax(dim=1)
#    )
  ]

lr_list=[0.0001,0.001,0.01,0.1,1,10,100]
weight_decay_list=[0.00000001,0.0000001,
                  0.000001,0.00001,0.0001,0.001,0.01,0.1,0,1,10]

for fil in filters:
    for LR in lr_list:
        for WD in weight_decay_list:
            train=trainer(_classfilter=copy.deepcopy(fil),_print=False,_lr=LR,_weight_decay=WD)#.to(device)
            train.train(dataloader_dict=gen_dataloader_dict(),_print=False,_num_epochs=100)
            train.graph("fil:{},lr={},weight_decay={}".format(fil,LR,WD))

In [ ]:
_print=True
dbprint=(lambda x:print(x)) if _print else (lambda x: None)
dbprint("11")

In [ ]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
output

In [ ]:
for a,b in dataloader_dict['train']:
    print(a.shape,b)
